In [11]:
import warnings
warnings.filterwarnings("ignore")
from joblib import parallel_backend
import multiprocessing

n_cores = multiprocessing.cpu_count()
import csv

# MILP 1

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import csv
import multiprocessing
from joblib import parallel_backend
import warnings
warnings.filterwarnings("ignore")
import multiprocessing
from joblib import parallel_backend

n_cores = multiprocessing.cpu_count()
print(f"Số lõi CPU có sẵn: {n_cores}")

# Danh sách các file dữ liệu
file_names = [
    'ionosphere.data',
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    "ionosphere_both_noise_outlier.txt"

]
# Tạo danh sách để lưu kết quả
results = []
print('Model: MILP1')
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header= None)

    # Tách dữ liệu và nhãn
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    # Các tham số của mô hình
    c = [1.0] * X.shape[1]
    l = [-2] * X.shape[1]  # Giới hạn dưới của trọng số wj
    u = [2] * X.shape[1]   # Giới hạn trên của trọng số wj
    C_values = [2**i for i in range(-3,6)]

    # Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho B
    best_B = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0

    # Danh sách các giá trị B cần kiểm tra
    n = X.shape[1]
    B_values = [i for i in range(1, n+1)]

    # for C in C_values:
    with parallel_backend('loky', n_jobs=n_cores):
        for B in B_values:
            for C in C_values:
                kf = KFold(n_splits=10, shuffle=True,random_state=42)
                cv_accuracies = []
                cv_AUC = []
                cv_times = []
                
                for train_index, val_index in kf.split(X):
                    start_time = time.time()
                    X_cv_train, X_cv_val = X[train_index], X[val_index]
                    y_cv_train, y_cv_val = y[train_index], y[val_index]
                    
                    X_cv_train = scaler.fit_transform(X_cv_train)
                    X_cv_val = scaler.transform(X_cv_val)

                    # Khởi tạo mô hình
                    opt_mod = Model(name='MILP1')

                    # Số lượng mẫu và số lượng thuộc tính
                    m, n = X_cv_train.shape

                    # Các biến quyết định
                    w = opt_mod.continuous_var_list(n, name='w')
                    b = opt_mod.continuous_var(name='b')
                    v = opt_mod.binary_var_list(n, name='v')
                    xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                    z = opt_mod.continuous_var_list(n,lb =0, name = 'z')

                    # Hàm mục tiêu
                    opt_mod.minimize(opt_mod.sum(xi[i] for i in range(m)))

                    # Ràng buộc
                    for i in range(m):
                        opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i,j] for j in range(n)) + b) >= 1 - xi[i])
                    for j in range(n):
                        opt_mod.add_constraint(w[j] <= v[j] * u[j])
                        opt_mod.add_constraint(w[j] >= l[j] * v[j])
                        opt_mod.add_constraint(w[j] >= -z[j])
                        opt_mod.add_constraint(w[j] <= z[j])
                
                    opt_mod.add_constraint(opt_mod.sum(c[j] * v[j] for j in range(n)) <= B)
                    
                    # Giải mô hình
                    solution = opt_mod.solve()

                    end_time = time.time()
                    fold_time = end_time - start_time
                    cv_times.append(fold_time)

                    if solution:
                        w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                        b_opt = solution.get_value('b')
                        v_opt = np.array([solution.get_value(f'v_{j}') for j in range(n)])
                        y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                        cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                        cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                        cv_accuracies.append(cv_accuracy)
                        cv_AUC.append(cv_auc)
                    

            # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị B
            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            # Cập nhật giá trị B tối ưu
            if mean_cv_AUC > best_mean_cv_AUC:
                best_B = B
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
                best_v = v_opt
        print(f'Best B value: {best_B}')
        print(f'Best C value: {best_C}')
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]


     
        print(f'v: {best_v}')
        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)

        result = {
            'Model': 'MILP1',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")



Số lõi CPU có sẵn: 20
Model: MILP1

Xử lý file: ionosphere.data
Not noise
Best B value: 2
Best C value: 32
Best Accuracy on 10-CV: 0.8803174603174602
Best AUC on 10-CV:  0.844513243399113
Average time per fold: 0.11 seconds
v: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Features selected: [1, 5]
Number of selected features: 2
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Best B value: 10
Best C value: 32
Best Accuracy on 10-CV: 0.8291269841269842
Best AUC on 10-CV:  0.7925595560046361
Average time per fold: 0.12 seconds
v: [1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Features selected: [1, 2, 3, 4, 5, 8, 9, 10, 21, 26]
Number of selected features: 10
----------------------------------------------------------------------------------------------------

Xử lý fil

# Pin - FS - SVM

In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import multiprocessing
from joblib import parallel_backend
import warnings
warnings.filterwarnings("ignore")

n_cores = multiprocessing.cpu_count()
print(f"Số lõi CPU có sẵn: {n_cores}")

# Danh sách các file dữ liệu
file_names = [
    'ionosphere.data',
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    "ionosphere_both_noise_outlier.txt"
]

print('Model: Pin-FS-SVM')
results = []
max_time_per_file = 30 * 60  # 30 phút

for iteration, file_name in enumerate(file_names):
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header=None)

    # Tách dữ liệu và nhãn
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    # Các tham số của mô hình
    l = [-2] * X.shape[1]
    u = [2] * X.shape[1]
    C_values = [2**i for i in range(-3, 6)]
    tau_values = [0.1, 0.5, 1]

    best_B = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    no_improvement_count = 0

    n = X.shape[1]
    B_values = [i for i in range(1, n+1)]

    start_file_time = time.time()

    with parallel_backend('loky', n_jobs=n_cores):
        for tau in tau_values:
            for C in C_values:
                for B in B_values:
                    kf = KFold(n_splits=10, shuffle=True, random_state=42)
                    cv_accuracies = []
                    cv_AUC = []
                    cv_times = []

                    for train_index, val_index in kf.split(X):
                        start_time = time.time()
                        X_cv_train, X_cv_val = X[train_index], X[val_index]
                        y_cv_train, y_cv_val = y[train_index], y[val_index]

                        # Chuẩn hóa dữ liệu
                        X_cv_train = scaler.fit_transform(X_cv_train)
                        X_cv_val = scaler.transform(X_cv_val)

                        # Khởi tạo mô hình
                        opt_mod = Model(name='Pin-FS-SVM')
                        m, n = X_cv_train.shape

                        # Các biến quyết định
                        w = opt_mod.continuous_var_list(n, name='w')
                        b = opt_mod.continuous_var(name='b')
                        v = opt_mod.binary_var_list(n, name='v')
                        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                        z = opt_mod.continuous_var_list(n, lb=0, name='z')

                        # Hàm mục tiêu
                        opt_mod.minimize(opt_mod.sum(z[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                        # Ràng buộc phân loại
                        for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) <= 1 + xi[i] * (1/tau))

                        # Ràng buộc tổng các đặc trưng được chọn
                        opt_mod.add_constraint(opt_mod.sum(v[j] for j in range(n)) <= B)

                        for j in range(n):
                            opt_mod.add_constraint(w[j] <= v[j] * u[j])
                            opt_mod.add_constraint(w[j] >= l[j] * v[j])
                            opt_mod.add_constraint(w[j] <= z[j])
                            opt_mod.add_constraint(w[j] >= -z[j])

                        # Giải mô hình
                        solution = opt_mod.solve()

                        end_time = time.time()
                        fold_time = end_time - start_time
                        cv_times.append(fold_time)

                        if solution:
                            w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                            b_opt = solution.get_value('b')
                            v_opt = np.array([solution.get_value(f'v_{j}') for j in range(n)])
                            y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                            cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                            cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                            cv_accuracies.append(cv_accuracy)
                            cv_AUC.append(cv_auc)
                        else:
                            print("Solution not found for fold.")

                    # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị B
                    mean_cv_accuracy = np.mean(cv_accuracies)
                    mean_cv_AUC = np.mean(cv_AUC)
                    mean_time_per_fold = np.mean(cv_times)

                    # Kiểm tra điều kiện early stopping
                    if mean_cv_AUC > best_mean_cv_AUC:
                        best_B = B
                        best_C = C
                        best_tau = tau
                        best_mean_cv_accuracy = mean_cv_accuracy
                        best_mean_cv_AUC = mean_cv_AUC
                        best_mean_time_per_fold = mean_time_per_fold
                        best_w = w_opt
                        best_b = b_opt
                        best_v = v_opt
                   
    print(f'Best B value: {best_B}')
    print(f'Best C: {best_C}')
    print(f'Best tau: {best_tau}')
    print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
    print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
    print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

    selected_features = [j + 1 for j in range(n) if best_w[j] != 0]

    print(f'v: {best_v}')
    print(f'Features selected: {selected_features}')
    print(f'Number of selected features: {len(selected_features)}')
    print('-' * 100)
    result = {
            'Model': 'Pin-FS-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C,
            'tau': best_tau
        }
    results.append(result)

 # Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C', 'tau']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")

Số lõi CPU có sẵn: 20
Model: Pin-FS-SVM

Xử lý file: ionosphere.data
Not noise
Best B value: 12
Best C: 2
Best tau: 0.1
Best Accuracy on 10-CV: 0.8860317460317461
Best AUC on 10-CV:  0.8548723740028088
Average time per fold: 0.13 seconds
v: [1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Features selected: [1, 3, 4, 5, 6, 7, 8, 10, 15, 21, 25, 31]
Number of selected features: 12
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Best B value: 14
Best C: 1
Best tau: 0.1
Best Accuracy on 10-CV: 0.8348412698412699
Best AUC on 10-CV:  0.8023922219085262
Average time per fold: 0.12 seconds
v: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
Features selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 18, 21, 26, 31]
Number of selected features: 14
----------------------------------------

# Pinball loss

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")
import warnings
warnings.filterwarnings("ignore")
from joblib import parallel_backend
import multiprocessing

n_cores = multiprocessing.cpu_count()
import csv

# Danh sách các file dữ liệu
file_names = [
    'ionosphere.data',
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    'ionosphere_both_noise_outlier.txt'
]
print('Model: Pin - SVM')
results = []
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header= None)

    # Tách dữ liệu và nhãn
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    C_values = [2**i for i in range(-3,6)]
    tau_values = [0.1, 0.5, 1]
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    best_C = None

    # Danh sách các giá trị B cần kiểm tra
    n = X.shape[1]
    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            for tau in tau_values:
                kf = KFold(n_splits=10, shuffle=True, random_state=42)
                cv_accuracies = []
                cv_AUC = []
                cv_times = []
                
                for train_index, val_index in kf.split(X):
                    start_time = time.time()
                    X_cv_train, X_cv_val = X[train_index], X[val_index]
                    y_cv_train, y_cv_val = y[train_index], y[val_index]
                    
                    X_cv_train = scaler.fit_transform(X_cv_train)
                    X_cv_val = scaler.transform(X_cv_val)

                    # Khởi tạo mô hình
                    opt_mod = Model(name='Pinball loss')

                    # Số lượng mẫu và số lượng thuộc tính
                    m, n = X_cv_train.shape

                    # Các biến quyết định
                    w = opt_mod.continuous_var_list(n, name='w')
                    b = opt_mod.continuous_var(name='b')
                    v = opt_mod.binary_var_list(n, name='v')
                    xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                    
                    # Hàm mục tiêu
                    opt_mod.minimize(0.5 * opt_mod.sum(w[j] ** 2 for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                    # Ràng buộc phân loại
                    for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                    for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) <= 1 + xi[i]*(1/tau))

                    
                    # Giải mô hình
                    solution = opt_mod.solve()

                    end_time = time.time()
                    fold_time = end_time - start_time
                    cv_times.append(fold_time)

                    if solution:
                        w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                        b_opt = solution.get_value('b')
                        y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                        cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                        cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                        cv_accuracies.append(cv_accuracy)
                        cv_AUC.append(cv_auc)
                    else:
                        print("Solution not found for fold.")

            # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị B
            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            # Cập nhật giá trị B tối ưu
            if mean_cv_accuracy > best_mean_cv_accuracy:
              
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_C = C
                best_b = b_opt
                best_tau = tau
        
        print('Best C: ', best_C)
        print('Best tau: ', best_tau)
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]

        print(f'Number of selected features: {len(list(best_w))}')
        print('-' * 100)
        result = {
            'Model': 'Pin-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C,
            'tau' : best_tau
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C', 'tau']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: Pinball loss - SVM

Xử lý file: ionosphere.data
Not noise
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.8719047619047618
Best AUC on 10-CV:  0.8255596034400382
Average time per fold: 0.10 seconds
Number of selected features: 34
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.8177777777777777
Best AUC on 10-CV:  0.7728085591944287
Average time per fold: 0.10 seconds
Number of selected features: 34
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_outlier.txt
Outlier
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.8149206349206348
Best AUC on 10-CV:  0.7741293072252825
Average time per fold: 0.10 seconds
Number of selected features: 34
----------------------------------------------------------------------------------------------------


# L1 - SVM

In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    'ionosphere.data',
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    'ionosphere_both_noise_outlier.txt'
]
print('Model: L1-SVM')
results = []
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header= None)

    # Tách dữ liệu và nhãn
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    # Các tham số của mô hình
    C_values = [2**i for i in range(-3, 6)]

    best_C = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0

    # Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho C
    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            cv_accuracies = []
            cv_AUC = []
            cv_times = []

            for train_index, val_index in kf.split(X):
                start_time = time.time()

                X_cv_train, X_cv_val = X[train_index], X[val_index]
                y_cv_train, y_cv_val = y[train_index], y[val_index]
                
                X_cv_train = scaler.fit_transform(X_cv_train)
                X_cv_val = scaler.transform(X_cv_val)

                # Khởi tạo mô hình
                opt_mod = Model(name='L1-SVM')

                # Số lượng mẫu và số lượng thuộc tính
                m, n = X_cv_train.shape

                # Các biến quyết định
                w = opt_mod.continuous_var_list(n, name='w')
                b = opt_mod.continuous_var(name='b')
                v = opt_mod.continuous_var_list(n, name='v', lb=0)
                xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

                # Hàm mục tiêu
                opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                # Ràng buộc
                for i in range(m):
                    opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                for j in range(n):
                    opt_mod.add_constraint(w[j] <= v[j])
                    opt_mod.add_constraint(-v[j] <= w[j])

                # Giải mô hình
                solution = opt_mod.solve()

                end_time = time.time()
                fold_time = end_time - start_time
                cv_times.append(fold_time)

                if solution:
                    w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                    b_opt = solution.get_value('b')
                    y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                    cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                    cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                    cv_accuracies.append(cv_accuracy)
                    cv_AUC.append(cv_auc)

            # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị C
            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            # Cập nhật giá trị C tối ưu
            if mean_cv_accuracy > best_mean_cv_accuracy:
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
       
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]


        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)
        result = {
            'Model': 'L1-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: L1-SVM

Xử lý file: ionosphere.data
Not noise
Best Accuracy on 10-CV: 0.8803174603174604
Best AUC on 10-CV:  0.8495495718050066
Average time per fold: 0.04 seconds
Features selected: [1, 3, 4, 5, 6, 7, 8, 9, 10, 15, 18, 21, 25, 31]
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Best Accuracy on 10-CV: 0.832063492063492
Best AUC on 10-CV:  0.7982268492377188
Average time per fold: 0.04 seconds
Features selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 18, 21, 23, 26, 31]
Number of selected features: 16
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_outlier.txt
Outlier
Best Accuracy on 10-CV: 0.8348412698412699
Best AUC on 10-CV:  0.810080074197351
Average time per fold: 0.05 seconds
Features selected: [1, 3, 4, 5, 7, 8, 9, 10, 15, 16, 18, 31]
Number of selected features

# L2 - SVM

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
   'ionosphere.data',
   'ionosphere_noise_label_feature.txt',
   'ionosphere_outlier.txt',
   'ionosphere_both_noise_outlier.txt'
]
print('Model: L2-SVM')
results = []
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header= None)

    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    

    # Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho B
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    best_C = None
    C_values = [2**i for i in range(-3, 6)]

    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            cv_accuracies = []
            cv_AUC = []
            cv_times = []

            for train_index, val_index in kf.split(X):
                start_time = time.time()

                X_cv_train, X_cv_val = X[train_index], X[val_index]
                y_cv_train, y_cv_val = y[train_index], y[val_index]
                
                X_cv_train = scaler.fit_transform(X_cv_train)
                X_cv_val = scaler.transform(X_cv_val)

                # Khởi tạo mô hình
                opt_mod = Model(name='L2-SVM')

                # Số lượng mẫu và số lượng thuộc tính
                m, n = X_cv_train.shape

                # Các biến quyết định
                w = opt_mod.continuous_var_list(n, name='w')
                b = opt_mod.continuous_var(name='b')
                xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

                # Hàm mục tiêu
                opt_mod.minimize(0.5 * opt_mod.sum(w[j] ** 2 for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                # Ràng buộc
                for i in range(m):
                    opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                
                # Giải mô hình
                solution = opt_mod.solve()

                end_time = time.time()
                fold_time = end_time - start_time
                cv_times.append(fold_time)

                if solution:
                    w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                    b_opt = solution.get_value('b')
                    y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                    cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                    cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                    cv_accuracies.append(cv_accuracy)
                    cv_AUC.append(cv_auc)
                    

            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            if mean_cv_accuracy > best_mean_cv_accuracy:
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
       
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]


      
        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)
        result = {
            'Model': 'L2-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: L2-SVM

Xử lý file: ionosphere.data
Not noise
Best Accuracy on 10-CV: 0.8831746031746033
Best AUC on 10-CV:  0.8543926453980802
Average time per fold: 0.06 seconds
Features selected: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Number of selected features: 33
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Best Accuracy on 10-CV: 0.834920634920635
Best AUC on 10-CV:  0.8023935159043856
Average time per fold: 0.06 seconds
Features selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Number of selected features: 34
----------------------------------------------------------------------------------------------------

Xử lý file: ionosphere_outlier.txt
Outlier
Best Accuracy on 10-CV: 0.8348412698412699
Best AUC on 10

# Fisher - SVM

In [18]:
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from docplex.mp.model import Model
import time
import multiprocessing
from joblib import parallel_backend
import warnings
warnings.filterwarnings("ignore")

n_cores = multiprocessing.cpu_count()
print(f"Số lõi CPU có sẵn: {n_cores}")

def calculate_f_score(X, y):
    pos_indices = y == 1
    neg_indices = y == -1
    
    X_positive = X[pos_indices]
    X_negative = X[neg_indices]
    
    n_positive = X_positive.shape[0]
    n_negative = X_negative.shape[0]
    
    pos_mean = np.mean(X_positive, axis=0)
    neg_mean = np.mean(X_negative, axis=0)
    total_mean = np.mean(X, axis=0)
    
    # Xử lý trường hợp phương sai bằng 0
    variance_positive = np.sum((X_positive - pos_mean)**2, axis=0) / (n_positive - 1)
    variance_negative = np.sum((X_negative - neg_mean)**2, axis=0) / (n_negative - 1)
    
    numerator = (pos_mean - total_mean)**2 + (neg_mean - total_mean)**2
    denominator = variance_positive + variance_negative
    
    # Xử lý trường hợp mẫu số bằng 0
    denominator = np.where(denominator == 0, np.inf, denominator)
    f_scores = numerator / denominator
    
    # Thay thế các giá trị NaN và inf bằng 0
    f_scores = np.nan_to_num(f_scores, nan=0.0, posinf=0.0, neginf=0.0)
    
    return f_scores

def svm_train_qp(X, y, C, lambda_val=1e-5):
        # Khởi tạo mô hình
        opt_mod = Model(name='L1-SVM')

        # Số lượng mẫu và số lượng thuộc tính
        m, n = X.shape

        # Các biến quyết định
        w = opt_mod.continuous_var_list(n, name='w')
        b = opt_mod.continuous_var(name='b')
        v = opt_mod.continuous_var_list(n, name='v', lb=0)
        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

        # Hàm mục tiêu
        opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

        # Ràng buộc
        for i in range(m):
            opt_mod.add_constraint(y[i] * (opt_mod.sum(w[j] * X[i, j] for j in range(n)) + b) >= 1 - xi[i])
        for j in range(n):
            opt_mod.add_constraint(w[j] <= v[j])
            opt_mod.add_constraint(-v[j] <= w[j])
        
        solution = opt_mod.solve()
        if solution:
            w_opt = np.array([solution.get_value(w[j]) for j in range(n)])
            b_opt = solution.get_value(b)
            return w_opt, b_opt
        return None, None

def feature_selection_with_f_score_cv(X, y, thresholds, C_values):
    f_scores = calculate_f_score(X, y)
    best_accuracy = 0 
    best_auc = 0
    best_threshold = None
    best_error = float('inf')
    best_C = None
    avg_fold_times = []
    best_selected_features = None
    
    start_time = time.time()
    max_time = 3600  # 1 hour limit
    
    for C in C_values:
        if time.time() - start_time > max_time:
            print("Đã đạt giới hạn 1 giờ")
            break
            
        for threshold in thresholds:
            # Chọn đặc trưng có F-score lớn hơn ngưỡng
            selected_features = f_scores >= threshold
            
            # Đảm bảo có ít nhất 1 đặc trưng được chọn
            if not np.any(selected_features):
                # Chọn top 10% đặc trưng có F-score cao nhất
                num_features = max(1, int(0.1 * len(f_scores)))
                top_indices = np.argsort(f_scores)[-num_features:]
                selected_features = np.zeros_like(f_scores, dtype=bool)
                selected_features[top_indices] = True
                
            X_selected = X[:, selected_features]
            
            errors = []
            for _ in range(5):
                X_train, X_valid, y_train, y_valid = train_test_split(X_selected, y, test_size=0.2)
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_valid_scaled = scaler.transform(X_valid)
                
                w, b = svm_train_qp(X_train_scaled, y_train, C=C)
                
                if w is None:
                    continue
                
                y_pred = np.sign(np.dot(X_valid_scaled, w) + b)
                error = 1 - accuracy_score(y_valid, y_pred)
                errors.append(error)
            
            if not errors:
                continue
                
            avg_error = np.mean(errors)
            if avg_error < best_error:
                best_error = avg_error
                best_threshold = threshold
                
        selected_features = f_scores >= best_threshold
        if not np.any(selected_features):
            num_features = max(1, int(0.1 * len(f_scores)))
            top_indices = np.argsort(f_scores)[-num_features:]
            selected_features = np.zeros_like(f_scores, dtype=bool)
            selected_features[top_indices] = True
            
        X_subset = X[:, selected_features]
        
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        cv_accuracies = []
        cv_aucs = []
        times = []
            
        for train_idx, val_idx in kf.split(X_subset):
            start_fold_time = time.time()
            X_train, X_val = X_subset[train_idx], X_subset[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

            w_opt, b_opt = svm_train_qp(X_train, y_train, C=C)
            if w_opt is None:
                continue
                
            y_pred = np.sign(np.dot(X_val, w_opt) + b_opt)
            cv_accuracies.append(accuracy_score(y_val, y_pred))
            cv_aucs.append(roc_auc_score(y_val, y_pred))
            times.append(time.time() - start_fold_time)
        
        if not cv_accuracies:
            continue
            
        mean_accuracy = np.mean(cv_accuracies)
        mean_auc = np.mean(cv_aucs)
        mean_time = np.mean(times)
        avg_fold_times.append(mean_time)
        
        if mean_auc > best_auc:
            best_accuracy = mean_accuracy
            best_auc = mean_auc
            best_threshold = threshold
            best_C = C
            best_selected_features = selected_features.copy()
    
    if not avg_fold_times:
        avg_time_per_fold = 0
    else:
        avg_time_per_fold = np.mean(avg_fold_times)
        
    if best_selected_features is None:
        # Chọn top 10% đặc trưng có F-score cao nhất
        num_features = max(1, int(0.1 * len(f_scores)))
        top_indices = np.argsort(f_scores)[-num_features:]
        best_selected_features = np.zeros_like(f_scores, dtype=bool)
        best_selected_features[top_indices] = True
        
    return best_selected_features, best_accuracy, best_auc, best_C, best_threshold, avg_time_per_fold

# Danh sách các file dữ liệu
file_names = [
    "ionosphere.data",
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    'ionosphere_both_noise_outlier.txt'
]

print('Model: Filter - SVM')
results = []
C_values = [2**i for i in range(-3, 6)]

for iteration, file_name in enumerate(file_names):
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    df = pd.read_csv(file_name, header=None)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    y = np.where(y == 'g', 1, -1)
    
    # Sử dụng ngưỡng nhỏ hơn
    thresholds = np.percentile(calculate_f_score(X, y), [25, 50, 75 ])  # Thay đổi ngưỡng
    selected_features, best_accuracy, best_auc, best_C, best_threshold, avg_time_per_fold = feature_selection_with_f_score_cv(X, y, thresholds, C_values)
    
    print("Accuracy tốt nhất:", best_accuracy)
    print("Các đặc trưng tốt nhất:", np.where(selected_features)[0] + 1)
    print('Số lượng features:', len(np.where(selected_features)[0]))
    print('AUC tốt nhất:', best_auc)
    print('C tốt nhất:', best_C)
    print('Threshold tốt nhất:', best_threshold)
    print('Average time per fold:', avg_time_per_fold)
    
    result = {
        'Model': 'Fisher - SVM',
        'Type of model': noise_type,
        'Accuracy': best_accuracy,
        'AUC': best_auc,
        'Time': avg_time_per_fold,
        'Features selected': ', '.join(map(str, np.where(selected_features)[0] + 1)),
        'Number of features': len(np.where(selected_features)[0]),
        'C': best_C
    }
    results.append(result)

output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")

Số lõi CPU có sẵn: 20
Model: Filter - SVM

Xử lý file: ionosphere.data
Not noise
Accuracy tốt nhất: 0.8775396825396825
Các đặc trưng tốt nhất: [ 1  3  5  7  9 21 29 31 33]
Số lượng features: 9
AUC tốt nhất: 0.8418792107650803
C tốt nhất: 0.25
Threshold tốt nhất: 0.0557269866646512
Average time per fold: 0.027537957827250163

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Accuracy tốt nhất: 0.8264285714285714
Các đặc trưng tốt nhất: [ 1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 21 23 25 27 29 31
 33]
Số lượng features: 25
AUC tốt nhất: 0.7912030397139093
C tốt nhất: 0.125
Threshold tốt nhất: 0.057126380613913896
Average time per fold: 0.02869849999745687

Xử lý file: ionosphere_outlier.txt
Outlier
Accuracy tốt nhất: 0.8290476190476189
Các đặc trưng tốt nhất: [ 1  3  5  7  9 11 13 14 15 16 18 21 23 27 29 31 33]
Số lượng features: 17
AUC tốt nhất: 0.7987546349307354
C tốt nhất: 0.5
Threshold tốt nhất: 0.061319845784391876
Average time per fold: 0.026877495977613658

Xử lý f

# RFE-SVM

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import multiprocessing
import time
import csv

n_cores = multiprocessing.cpu_count()
print(f"Số lõi CPU có sẵn: {n_cores}")

# Danh sách các file dữ liệu
file_names = [
    'ionosphere.data',
    'ionosphere_noise_label_feature.txt',
    'ionosphere_outlier.txt',
    "ionosphere_both_noise_outlier.txt"
]

# Tạo danh sách để lưu kết quả
results = []
print('Model: RFE2-SVM')

for iteration, file_name in enumerate(file_names):
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
    else:
        noise_type = 'Noise + Outlier'
    
    # Đọc file
    df = pd.read_csv(file_name, header=None)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    # Chuyển đổi nhãn từ B/M thành -1/1
    y = np.where(y == 'g', 1, -1)

    # SVM huấn luyện sử dụng Quadratic Programming (QP)
    def svm_train_qp(X, y, C, lambda_val=1e-5):
        # Khởi tạo mô hình
        opt_mod = Model(name='L1-SVM')

        # Số lượng mẫu và số lượng thuộc tính
        m, n = X.shape

        # Các biến quyết định
        w = opt_mod.continuous_var_list(n, name='w')
        b = opt_mod.continuous_var(name='b')
        v = opt_mod.continuous_var_list(n, name='v', lb=0)
        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

        # Hàm mục tiêu
        opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

        # Ràng buộc
        for i in range(m):
            opt_mod.add_constraint(y[i] * (opt_mod.sum(w[j] * X[i, j] for j in range(n)) + b) >= 1 - xi[i])
        for j in range(n):
            opt_mod.add_constraint(w[j] <= v[j])
            opt_mod.add_constraint(-v[j] <= w[j])
        
        solution = opt_mod.solve()
        if solution:
            w_opt = np.array([solution.get_value(w[j]) for j in range(n)])
            b_opt = solution.get_value(b)
            return w_opt, b_opt
        return None, None

    # Hàm SVM-RFE với tìm kiếm Cross-Validation và tối ưu C
    def svm_rfe_with_cv(X, y, C_values):
        s = list(range(X.shape[1]))  # Các đặc trưng ban đầu
        best_result = {'accuracy': 0, 'auc': 0, 'num_features': len(s), 'features': s[:], 'C': None, 'avg_time_per_fold': 0}
        
        for C in C_values:
            s_temp = s[:]  # Bắt đầu với tất cả các đặc trưng ban đầu trong mỗi vòng lặp của C
            while len(s_temp) > 1:
                # Lấy các đặc trưng hiện tại trong s_temp
                X_subset = X[:, s_temp]
                w_opt, b_opt = svm_train_qp(X_subset, y, C)
                if w_opt is None:
                    break

                # Đánh giá mô hình bằng Cross-Validation
                cv_accuracies, cv_aucs = [], []
                times = []
                kf = KFold(n_splits=10, shuffle=True, random_state=42)
                for train_idx, val_idx in kf.split(X_subset):
                    start_time = time.time()
                    X_train, X_val = X_subset[train_idx], X_subset[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    # Chuẩn hóa
                    scaler = StandardScaler()
                    X_train = scaler.fit_transform(X_train)
                    X_val = scaler.transform(X_val)

                    w_opt, b_opt = svm_train_qp(X_train, y_train, C)
                    if w_opt is None:
                        continue

                    y_pred = np.sign(np.dot(X_val, w_opt) + b_opt)
                    cv_accuracies.append(accuracy_score(y_val, y_pred))
                    cv_aucs.append(roc_auc_score(y_val, y_pred))
                    end_time = time.time()
                    times.append(end_time - start_time)

                mean_accuracy = np.mean(cv_accuracies)
                mean_auc = np.mean(cv_aucs)
                mean_time_per_fold = np.mean(times)

                # Cập nhật kết quả tốt nhất cho giá trị C hiện tại
                if mean_auc > best_result['auc']:
                    best_result.update({
                        'accuracy': mean_accuracy,
                        'auc': mean_auc,
                        'num_features': len(s_temp),
                        'features': s_temp[:],
                        'C': C,
                        'avg_time_per_fold': mean_time_per_fold
                    })
                #Loại bỏ ít đặc trưng nhất
                num_to_remove = len(s_temp) // 2 
                least_important_features = np.argsort(w_opt ** 2)[:num_to_remove]
                s_temp = [s_temp[i] for i in range (len(s_temp)) if i not in least_important_features]

        return best_result

    # Danh sách các giá trị C để tìm kiếm
    C_values = [2 ** i for i in range(-3, 6)]
    
    # Gọi hàm SVM-RFE với tìm kiếm giá trị C tối ưu
    best_result = svm_rfe_with_cv(X, y, C_values)

    # In kết quả
    print("Số lượng đặc trưng tốt nhất:", best_result['num_features'])
    print("Accuracy tốt nhất:", best_result['accuracy'])
    print("Các đặc trưng tốt nhất:", list( i + 1 for i in best_result['features'] ))
    print("AUC tốt nhất:", best_result['auc'])
    print("Giá trị C tốt nhất:", best_result['C'])
    print("Thời gian trung bình mỗi fold:", best_result['avg_time_per_fold'])
    
    results.append({
        'Model': 'RFE-SVM',
        'Type of model': noise_type,
        'Accuracy': best_result['accuracy'],
        'AUC': best_result['auc'],
        'Time': best_result['avg_time_per_fold'],
        'Features selected': ', '.join(map(str, list( i + 1 for i in best_result['features'] ))),
        'Number of features': best_result['num_features'],
        'C': best_result['C']
    })

output_file = 'model_results_ionosphere.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)
print(f"Kết quả đã được lưu vào file {output_file}")

Số lõi CPU có sẵn: 20
Model: RFE2-SVM

Xử lý file: ionosphere.data
Not noise
Số lượng đặc trưng tốt nhất: 9
Accuracy tốt nhất: 0.8860317460317461
Các đặc trưng tốt nhất: [1, 3, 4, 5, 7, 8, 10, 21, 25]
AUC tốt nhất: 0.8588922671531367
Giá trị C tốt nhất: 0.25
Thời gian trung bình mỗi fold: 0.034516429901123045

Xử lý file: ionosphere_noise_label_feature.txt
Noise
Số lượng đặc trưng tốt nhất: 9
Accuracy tốt nhất: 0.8434126984126985
Các đặc trưng tốt nhất: [1, 2, 3, 4, 5, 7, 8, 9, 21]
AUC tốt nhất: 0.8107268492377188
Giá trị C tốt nhất: 4
Thời gian trung bình mỗi fold: 0.03275148868560791

Xử lý file: ionosphere_outlier.txt
Outlier
Số lượng đặc trưng tốt nhất: 9
Accuracy tốt nhất: 0.8433333333333334
Các đặc trưng tốt nhất: [1, 3, 4, 5, 7, 8, 10, 15, 31]
AUC tốt nhất: 0.8183433795194798
Giá trị C tốt nhất: 0.5
Thời gian trung bình mỗi fold: 0.02580575942993164

Xử lý file: ionosphere_both_noise_outlier.txt
Số lượng đặc trưng tốt nhất: 9
Accuracy tốt nhất: 0.7723015873015873
Các đặc trưng t